In [2]:
from SpotifySong import SpotifyDataProcessor

cid = '0b8a840537a24112b19217f9d194980c'
secret = '5756cefe5da347d99deea997e266ff98'
redirect_uri = "https://github.com/YujieGong/sds-final-project"
scope = 'user-library-read'
spotify_processor = SpotifyDataProcessor(client_id=cid, client_secret=secret, redirect_uri=redirect_uri, scope=scope)
df_song_result = spotify_processor.process_spotify_data()
print(df_song_result)

   index               name                 artist
0      1             Mostro  Los Amigos Invisibles
1      2          Stay Here                 Marizu
2      3  Deeper and Deeper          Jackie Mittoo
3      4            Mercado           BaianaSystem
4      5            7 Years           Eric Lumiere
5      6             Stones       Front Porch Step
6      7           Unlonely             Jason Mraz
7      8      Setting Forth           Eddie Vedder
8      9    I Won't Give Up             Jason Mraz
9     10        Stand by Me           Boyce Avenue


In [ ]:
from SpotifySong import song

def main():
    #import data and use the class for song recommendations
    user_data = pd.read_csv("top_10_songs.csv")
    #user_data = pd.read_csv("top_10_songs_dataset2.csv")
    name_list = user_data["name"].tolist()
    artist_list = user_data["artist"].tolist()
    for i in range(10):
        new_song = song(name_list[i],artist_list[i])
        new_song.music_recommendation()    

if __name__ == "__main__":
    main()

All The World (I Tell Myself)---Correatown
Stand Your Ground---Joshua Hyslop
Heaven Knows---Five For Fighting
What Would I Do Without You---Drew Holcomb & The Neighbors
Don't Wanna Know - Acoustic Version---The Mayries
Mirrors---Boyce Avenue;Fifth Harmony
Always Be My Baby---Rosie Thomas;The Shins;Sufjan Stevens;Josh Ottum
End Of The Road---Eddie Vedder
